In [1]:
import numpy as np
import pandas as pd
import os, time, re
import pickle, gzip, datetime

from datetime import datetime

now = datetime.now()

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl
%matplotlib inline


from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, mean_squared_error
import lightgbm as lgb


import tensorflow as tf
import keras

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Input, Lambda
from tensorflow.keras.layers import Embedding, Flatten, dot
from tensorflow.keras import regularizers
from tensorflow.keras.losses import mse, binary_crossentropy

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2025-10-14 11:44:06.360923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760442246.384820    5081 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760442246.392327    5081 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/movie-ratings/ratings.csv


In [2]:
df=pd.read_csv("/kaggle/input/movie-ratings/ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 1.0 GB


In [4]:
df.shape

(33832162, 4)

In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [7]:
#rating distribution across the dataset
# in real world setting it is obvious for the data to be skewed towards higher ratings

df1=df["rating"].value_counts()
df1=df1.reset_index()
df1["proportion"]=df1["count"]*100/len(df)
df1

,rating,count,proportion
0,4.0,8835955,26.117027
1,3.0,6400664,18.918874
2,5.0,4957902,14.654405
3,3.5,4465001,13.197504
4,4.5,3123055,9.231024
5,2.0,2146492,6.344531
6,2.5,1760733,5.204317
7,1.0,1013645,2.996099
8,0.5,566306,1.673869
9,1.5,562409,1.662350


In [8]:
#number of users
n_users_org=df["userId"].nunique()

In [9]:
#number of movies
n_movies_org=df["movieId"].nunique()

In collaborative filtering, you usually have a user–item interaction matrix:
Rows = users
Columns = items (movies/products)
Entries = rating (explicit) or interaction (implicit: click, view, purchase)

Most users don’t rate most items → so most entries are empty (missing).
That’s sparsity.

Formally:
sparsity = 1 − (number of observed ratings/number of possible ratings)

Our dataset is sparse almost entirely (>99%). 

Why sparsity matters
- High sparsity → harder to learn similarities (cold-start users/items).
- Too sparse → overfitting risk (not enough overlap).
- Dense subset (like a 1k×1k sample) → useful for prototyping.

To reduce this sparsity, will take a sample of movies/users with greater number of ratings/rated highest number of times(highest interactions), thereby overcoming:
1. Long tailed problem - where movies with 1 or 2 ratings will be ignored
2. Hence, avoiding cold start problem - will have no ratings to learn from
3. Hence, avoiding high sparsity

In [10]:
#calculating sparsity
1-(len(df["rating"])/(df["userId"].nunique()*df["movieId"].nunique()))

0.9987719735903972

1. Movies are the "content backbone"
Some movies are rated thousands of times (popular blockbusters).
Others are rated only once or twice (obscure indie films).

If we include those rare movies, our matrix becomes very sparse and cold-start dominates.

=> That’s why we forcefully pick the top 1000 movies — we want only those with strong collaborative signal.

2. Users are more variable
We don’t want to always pick the same top 1000 most-active users.
Why? Because then your subset would be biased toward power users who rate hundreds or thousands of movies.

That subset wouldn’t reflect the variety of user behaviors (light raters vs. heavy raters).

=> So instead, you take a random sample of 1000 users (from the set of active users).

This balances the dataset.
=> Keeps diversity in user behaviors.
=> Makes sure you don’t just overfit to super-raters.

3. The balance

So:
Movies → fix the set to the most rated ones (dense overlap).
Users → sample a representative group (avoid bias, retain variety).

This creates a dense yet balanced subset.

4. What if we didn’t do this?
If we randomly sample movies → we’ll likely include many with just 1–2 ratings → kills overlap, matrix too sparse.
If we take only the very top users → our dataset only reflects “power raters”, and recommendations won’t generalize well.

So this asymmetric choice is a practical hack:
Items: deterministic top-k → ensures enough data per item => to guarantee density (avoid long-tail)
Users: random sample from active pool → ensures variety =>to preserve diversity and prevent bias toward only power-users.

In [11]:
#top 1000 highly rated movies (highest number of times)
movieIndex=df.groupby("movieId").count().sort_values(by="rating",ascending=False)[0:1000].index
movieIndex

ratingDF=df[df['movieId'].isin(movieIndex)]
ratingDF.count()

userId       20152541
movieId      20152541
rating       20152541
timestamp    20152541
dtype: int64

In [12]:
#top 1000 highest interactions
userIndex=ratingDF.groupby("userId").count().sort_values(by="rating",ascending=False).sample(n=1000, random_state=2018).index
userIndex

ratingDF1=ratingDF[ratingDF['userId'].isin(userIndex)]
ratingDF1.count()

userId       66616
movieId      66616
rating       66616
timestamp    66616
dtype: int64

In [13]:
#reindexing from 1 to 1000
movies=ratingDF1.movieId.unique()
moviesDF=pd.DataFrame(movies, columns=["originalMovieId"])
moviesDF["newMovieId"]=moviesDF.index+1
moviesDF

users=ratingDF1.userId.unique()
usersDF=pd.DataFrame(users,columns=["originalUserId"])
usersDF["newUserId"]=usersDF.index+1
usersDF

,originalUserId,newUserId
0,142,1
1,561,2
2,833,3
3,1590,4
4,1621,5
...,...,...
995,328497,996
996,329482,997
997,329982,998
998,330365,999


In [14]:
#merging dataframes
ratingDF1=ratingDF1.merge(moviesDF, left_on="movieId", right_on="originalMovieId")
ratingDF1

,userId,movieId,rating,timestamp,originalMovieId,newMovieId
0,142,32,3.5,1164321319,32,1
1,142,104,3.5,1164427217,104,2
2,142,150,3.5,1164427135,150,3
3,142,223,3.5,1164426847,223,4
4,142,253,3.0,1164321325,253,5
...,...,...,...,...,...,...
66611,330672,81845,4.5,1435885144,81845,721
66612,330672,96610,1.0,1436575287,96610,833
66613,330672,97304,4.5,1435885444,97304,733
66614,330672,104841,0.5,1435885430,104841,814


In [15]:
ratingDF1.drop("originalMovieId", inplace=True, axis=1)

In [16]:
ratingDF1=ratingDF1.merge(usersDF, left_on="userId", right_on="originalUserId")
ratingDF1.head()

,userId,movieId,rating,timestamp,newMovieId,originalUserId,newUserId
0,142,32,3.5,1164321319,1,142,1
1,142,104,3.5,1164427217,2,142,1
2,142,150,3.5,1164427135,3,142,1
3,142,223,3.5,1164426847,4,142,1
4,142,253,3.0,1164321325,5,142,1


In [17]:
ratingDF1.drop("originalUserId", inplace=True, axis=1)

In [18]:
ratingDF1

,userId,movieId,rating,timestamp,newMovieId,newUserId
0,142,32,3.5,1164321319,1,1
1,142,104,3.5,1164427217,2,1
2,142,150,3.5,1164427135,3,1
3,142,223,3.5,1164426847,4,1
4,142,253,3.0,1164321325,5,1
...,...,...,...,...,...,...
66611,330672,81845,4.5,1435885144,721,1000
66612,330672,96610,1.0,1436575287,833,1000
66613,330672,97304,4.5,1435885444,733,1000
66614,330672,104841,0.5,1435885430,814,1000


In [19]:
1-(len(ratingDF1["rating"])/(ratingDF1["userId"].nunique()*ratingDF1["movieId"].nunique()))

0.933384

In [20]:
n_users=ratingDF1["userId"].nunique() 
n_movies=ratingDF1["movieId"].nunique() 
n_ratings=len(ratingDF1)

n_users, n_movies, n_ratings

(1000, 1000, 66616)

In [21]:
#generating train, test and val sets
train_set, test_set=train_test_split(ratingDF1, test_size=0.2, shuffle=True, random_state=42)
train_set, val_set=train_test_split(train_set, test_size=0.2, shuffle=True, random_state=42)
train_set.shape, val_set.shape, test_set.shape

((42633, 6), (10659, 6), (13324, 6))

In [22]:
# Ratings matrix for train
ratings_train = np.zeros((n_users, n_movies))
for row in train_set.itertuples():
    ratings_train[row[6]-1, row[5]-1] = row[3]

In [23]:
ratings_train

array([[3.5, 3.5, 3.5, ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [24]:
sparsity = float(len(ratings_train .nonzero()[0]))
sparsity = sparsity/(ratings_train.shape[0] * ratings_train.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

Sparsity: 4.26%


In [25]:
#Ratings matrix for validation
ratings_val=np.zeros((n_users, n_movies))
for row in val_set.itertuples():
    ratings_val[row[6]-1, row[5]-1]=row[3]

In [26]:
#Ratings matrix for test set
ratings_test=np.zeros((n_users, n_movies))
for row in test_set.itertuples():
    ratings_test[row[6]-1,row[5]-1]=row[3]

In [27]:
actual_val=ratings_val[ratings_val.nonzero()].flatten()
len(actual_val)

10659

In [28]:
pred_val = np.zeros((len(val_set),1))
pred_val[pred_val==0] = 3.5
len(pred_val)

10659

In [29]:
#baseline experimentation, mse from 3.5, assumes everyone rates around the same global average.
print("MSE: ", mean_squared_error(pred_val, actual_val))

MSE:  1.1327751196172249


In [30]:
#ratings matrix for val predictions
ratings_val_pred_user = np.zeros((n_users, n_movies))
i = 0
global_mean=np.mean(ratings_train[ratings_train>0])
for row in ratings_train:
    # print(row)
    ratings_val_pred_user[i][ratings_val_pred_user[i]==0] = np.mean(row[row>0]) if np.any(row > 0) else global_mean
    i += 1

In [31]:
pred_val_ratings_user=ratings_val_pred_user[ratings_val.nonzero()].flatten()

In [32]:
print(np.isnan(pred_val_ratings_user).sum(), np.isnan(actual_val).sum())


0 0


In [33]:
#assumes different users have different typical ratings, but every movie is equally likable.
#Captures user bias, ignores movie bias.
user_avg=mean_squared_error(pred_val_ratings_user, actual_val)
print("MSE using user avg: ", user_avg)

MSE using user avg:  0.8813083450888654


In [34]:
ratings_val_pred_movie = np.zeros((n_users, n_movies)).T
i = 0
global_mean=np.mean(ratings_train[ratings_train>0])
for row in ratings_train.T:
    # print(row)
    ratings_val_pred_movie[i][ratings_val_pred_movie[i]==0] = np.mean(row[row>0]) if np.any(row > 0) else global_mean
    i += 1

In [35]:
ratings_val_pred_movie=ratings_val_pred_movie.T
pred_val_ratings_movie=ratings_val_pred_movie[ratings_val.nonzero()].flatten()

In [36]:
movie_avg=mean_squared_error(pred_val_ratings_movie, actual_val)
print("MSE using movie avg: ", movie_avg)

MSE using movie avg:  0.9914209541409421


In [37]:
n_latent_factors = 1
user_input = Input(shape=[1], name='user')
user_embedding = Embedding(input_dim=n_users + 1, output_dim=n_latent_factors,name='user_embedding')(user_input)
user_vec = Flatten(name='flatten_users')(user_embedding)

movie_input = Input(shape=[1], name='movie')
movie_embedding = Embedding(input_dim=n_movies + 1,output_dim=n_latent_factors,name='movie_embedding')(movie_input)
movie_vec = Flatten(name='flatten_movies')(movie_embedding)

product = dot([movie_vec, user_vec], axes=1)

model = Model(inputs=[user_input, movie_input], outputs=product)
model.compile('adam', 'mean_squared_error')

I0000 00:00:1760442282.829843    5081 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1760442282.830655    5081 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [38]:
history = model.fit(x=[train_set.newUserId, train_set.newMovieId],
                    y=train_set.rating, epochs=100,
                    validation_data=([val_set.newUserId,
                                      val_set.newMovieId], val_set.rating),
                    verbose=1)

Epoch 1/100


I0000 00:00:1760442284.645564    5147 service.cc:148] XLA service 0x7e56f8004d80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760442284.645615    5147 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760442284.645621    5147 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1760442284.716202    5147 cuda_dnn.cc:529] Loaded cuDNN version 90300


  77/1333 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 14.4037

I0000 00:00:1760442284.939173    5147 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1333/1333 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 14.2375 - val_loss: 13.5812
Epoch 2/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 12.8786 - val_loss: 10.7045
Epoch 3/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.7812 - val_loss: 7.5649
Epoch 4/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 6.8127 - val_loss: 5.1606
Epoch 5/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4.6390 - val_loss: 3.6884
Epoch 6/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3.3349 - val_loss: 2.8051
Epoch 7/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2.5138 - val_loss: 2.2328
Epoch 8/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.9940 - val_loss: 1.8441
Epoch 9/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.6627 - val_loss: 1.5746
Epoch 10/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.4306 - val_loss: 1.3857
Epoch 11/100
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2525 - val_loss: 1.2507
Epoch 12/100
1333/1333 ━━━━